# கிளவுடில் தரவியல் அறிவியல்: "Azure ML SDK" முறையில் 

## அறிமுகம்

இந்த நோட்புக்கில், Azure ML SDK-ஐ பயன்படுத்தி ஒரு மாதிரியை பயிற்சி, பிரசாரம் மற்றும் பயன்பாடு செய்வது எப்படி என்பதை நாம் கற்றுக்கொள்வோம்.

முன்னேற்பாடுகள்:
1. நீங்கள் Azure ML வேலைவாய்ப்பகத்தை உருவாக்கியுள்ளீர்கள்.
2. நீங்கள் [Heart Failure dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data)-ஐ Azure ML-க்கு ஏற்றியுள்ளீர்கள்.
3. நீங்கள் இந்த நோட்புக்கை Azure ML Studio-க்கு பதிவேற்றியுள்ளீர்கள்.

அடுத்த படிகள்:

1. ஏற்கனவே உள்ள வேலைவாய்ப்பகத்தில் ஒரு பரிசோதனை உருவாக்கவும்.
2. ஒரு கணினி களத்தை உருவாக்கவும்.
3. தரவுத்தொகுப்பை ஏற்றவும்.
4. AutoMLConfig-ஐ பயன்படுத்தி AutoML-ஐ அமைக்கவும்.
5. AutoML பரிசோதனையை இயக்கவும்.
6. முடிவுகளை ஆராய்ந்து சிறந்த மாதிரியை பெறவும்.
7. சிறந்த மாதிரியை பதிவு செய்யவும்.
8. சிறந்த மாதிரியை பிரசாரம் செய்யவும்.
9. இறுதிப்புள்ளியை பயன்படுத்தவும்.

## Azure Machine Learning SDK-க்கு குறிப்பிட்ட இறக்குமதிகள்


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## வேலைநிலையை ஆரம்பிக்கவும்  
நிலைப்படுத்தப்பட்ட கட்டமைப்பிலிருந்து ஒரு வேலைநிலை பொருளை ஆரம்பிக்கவும். config.json கோப்பு .\config.json இடத்தில் இருப்பதை உறுதிப்படுத்தவும்.


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Azure ML பரிசோதனை உருவாக்கவும்

நாம் இப்போது தொடங்கிய வேலைத்தளத்தில் 'aml-experiment' என்ற பெயரில் ஒரு பரிசோதனையை உருவாக்குவோம்.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## கணினி கிளஸ்டர் உருவாக்கவும்  
உங்கள் AutoML இயக்கத்திற்காக ஒரு [கணினி இலக்கு](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) உருவாக்க வேண்டும்.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## தரவுகள்
தரவுத்தொகுப்பை Azure ML-க்கு பதிவேற்றியுள்ளீர்கள் மற்றும் முக்கியம் தரவுத்தொகுப்பின் பெயருடன் ஒரே மாதிரியானதாக இருக்கிறது என்பதை உறுதிப்படுத்துங்கள்.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## ஆட்டோஎம்எல் கட்டமைப்பு


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## ஆட்டோஎம்.எல் இயக்கம்


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

## சிறந்த மாதிரியை சேமிக்கவும்


In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## சிறந்த மாதிரியை பிரசுரிக்கவும்

சிறந்த மாதிரியை பிரசுரிக்க கீழே உள்ள குறியீட்டை இயக்கவும். பிரசுரத்தின் நிலையை Azure ML போர்டலில் காணலாம். இந்த படி சில நிமிடங்கள் ஆகலாம்.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## முடிவுநிலைப் புள்ளியை பயன்படுத்துதல்  
நீங்கள் கீழே உள்ள உள்ளீடு மாதிரியில் உள்ளீடுகளைச் சேர்க்கலாம்.  


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கிறோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாகக் கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
